# Pipeline Architecture with idspy

This notebook demonstrates how to build and customize pipelines. Pipelines orchestrate sequences of steps with built-in event handling, conditional execution, and ML-specific features.

## What you'll learn

In this tutorial, you'll discover how to:

1. **Build Basic Pipelines** - Orchestrate sequences of steps
2. **Create Custom Steps** - Define reusable components
3. **Handle Events** - Add monitoring and debugging capabilities  
4. **Use Fittable Pipelines** - ML workflows with separate fit/transform phases
5. **Priority Systems** - Control execution order with multi-priority hooks
6. **Repeatable Pipelines** - Conditional re-execution based on storage state

## Key Benefits

- **Modular Design**: Compose complex workflows from simple steps
- **Event Integration**: Built-in observability and debugging
- **ML-Ready**: Fittable pipelines for training/inference workflows
- **Flexible Execution**: Conditional and repeatable pipeline patterns
- **State Management**: Automatic storage integration for persistence

---

Let's start by setting up our environment and building our first pipeline.

In [ ]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [18]:
from src.idspy.core.storage.dict import DictStorage
from src.idspy.core.step.base import Step
from src.idspy.core.step.fittable import FittableStep
from src.idspy.core.pipeline.base import Pipeline, PipelineEvent
from src.idspy.core.pipeline.fittable import FittablePipeline
from src.idspy.core.pipeline.observable import ObservablePipeline
from src.idspy.core.pipeline.repeatable import RepeatablePipeline, StoragePredicate
from src.idspy.core.events.bus import EventBus
from src.idspy.core.events.event import Event

## Building Blocks - Example Steps

Before building pipelines, let's define some example processing steps. Each step declares its input/output requirements through the `bindings()` method and implements its logic in `compute()`:

In [19]:
# Define example steps for our pipelines
class Load(Step):
    """Loads initial data."""
    def bindings(self):
        return {"data": "data"}

    def compute(self):
        # Simulate loading some data
        return {"data": [1, 2, 3, 4, 5]}

@Step.needs("data")
class Sum(Step):
    """Calculates sum of input data."""
    def bindings(self):
        return {"sum": "sum", "data": "data"}

    def compute(self, data):
        total = sum(data)
        print(f"  Sum of {data} = {total}")
        return {"sum": total}

## Basic Pipeline Execution

Let's start with a simple pipeline that loads data and calculates its sum:

In [20]:
# Create and run a basic pipeline
storage = DictStorage()
pipeline = Pipeline([Load(), Sum()], name="BasicPipeline", storage=storage)

print("Running Basic Pipeline")
print("=" * 40)

result = pipeline.run()

print(f"Pipeline result: {result}")

print("\nThe pipeline automatically manages data flow between steps!")

Running Basic Pipeline
  Sum of [1, 2, 3, 4, 5] = 15
Pipeline result: {'data': [1, 2, 3, 4, 5], 'sum': 15}

The pipeline automatically manages data flow between steps!


## Custom Pipeline with Event Hooks

You can create custom pipeline classes that respond to lifecycle events. This is perfect for adding logging, monitoring, or custom behaviors:

In [21]:
# Create a custom pipeline with event hooks
class MyPipeline(Pipeline):
    @Pipeline.hook(PipelineEvent.PIPELINE_START)
    def _start(self) -> None:
        print("[PIPELINE] Starting custom pipeline execution!")

    @Pipeline.hook(PipelineEvent.STEP_START)
    def _before_step(self, step: Step, index: int, inputs: dict) -> None:
        print(f"[STEP {index+1}] Starting: {step.__class__.__name__}")
        print(f"\tInputs: {list(inputs.keys())}")

    @Pipeline.hook(PipelineEvent.STEP_END)
    def _after_step(self, step: Step, index: int, outputs: dict) -> None:
        print(f"[STEP {index+1}] Completed: {step.__class__.__name__}")
        print(f"\tOutputs: {list(outputs.keys())}")

    @Pipeline.hook(PipelineEvent.PIPELINE_END)
    def _finish(self, result: dict) -> None:
        print("[PIPELINE] Custom pipeline execution completed!")
        print(f"\tFinal result keys: {list(result.keys())}")

# Test the custom pipeline
print("Testing Custom Pipeline with Event Hooks")
print("=" * 50)

storage = DictStorage()
custom_pipeline = MyPipeline([Load(), Sum()], name="CustomPipeline", storage=storage)
custom_pipeline.run()

print(f"\nFinal storage state: {storage.as_dict()}")

Testing Custom Pipeline with Event Hooks
[PIPELINE] Starting custom pipeline execution!
[STEP 1] Starting: Load
	Inputs: []
[STEP 1] Completed: Load
	Outputs: ['data']
[STEP 2] Starting: Sum
	Inputs: ['data']
  Sum of [1, 2, 3, 4, 5] = 15
[STEP 2] Completed: Sum
	Outputs: ['sum']
[PIPELINE] Custom pipeline execution completed!
	Final result keys: ['data', 'sum']

Final storage state: {'data': [1, 2, 3, 4, 5], 'sum': 15}


## Fittable Pipelines

Automatic fitting lifecycle for ML workflows. Controls refit behavior across runs.

In [22]:
@Step.needs("data")
class MeanCenter(FittableStep):
    """Centers data by subtracting the mean."""
    def __init__(self):
        super().__init__(self)
        self.mean = None

    def bindings(self):
        return {"data": "data", "centered_data": "centered_data"}

    def fit_impl(self, data):
        self.mean = sum(data) / len(data)
        print(f"  Fitted mean: {self.mean}")
        return {"mean": self.mean}

    def compute(self, data):
        centered = [x - self.mean for x in data]
        print(f"  Centered data: {centered}")
        return {"centered_data": centered}

In [23]:
storage = DictStorage({"data": [1.0, 2.0, 3.0]})
pipeline = FittablePipeline([MeanCenter(), Sum()], name="FitPipe", refit=False, storage=storage)
pipeline.run()
print("After first run:", storage.as_dict())
print("Mean learned:", pipeline._steps[0].mean)
# After first run: {'data': [-1.0, 0.0, 1.0], 'sum': 0}
# Mean learned: 2.0

# Second run without refit (uses same mean=2.0)
storage.set({"data": [2.0, 4.0, 6.0]})
pipeline.run()
print("Second run (no refit):", storage.as_dict())
print("Mean still:", pipeline._steps[0].mean)
# Second run (no refit): {'data': [0.0, 2.0, 4.0], 'sum': 6}
# Mean still: 2.0

# Pipeline with refit=True (learns new mean=4.0)
storage.set({"data": [2.0, 4.0, 6.0]})
pipeline = FittablePipeline([MeanCenter(), Sum()], name="FitPipeRefit", refit=True, storage=storage)
pipeline.run()
print("With refit=True:", storage.as_dict())
print("New mean learned:", pipeline._steps[0].mean)
# With refit=True: {'data': [-2.0, 0.0, 2.0], 'sum': 0}
# New mean learned: 4.0

  Fitted mean: 2.0
  Centered data: [-1.0, 0.0, 1.0]
  Sum of [1.0, 2.0, 3.0] = 6.0
After first run: {'data': [1.0, 2.0, 3.0], 'centered_data': [-1.0, 0.0, 1.0], 'sum': 6.0}
Mean learned: 2.0
  Centered data: [0.0, 2.0, 4.0]
  Sum of [2.0, 4.0, 6.0] = 12.0
Second run (no refit): {'data': [2.0, 4.0, 6.0], 'centered_data': [0.0, 2.0, 4.0], 'sum': 12.0}
Mean still: 2.0
  Fitted mean: 4.0
  Centered data: [-2.0, 0.0, 2.0]
  Sum of [2.0, 4.0, 6.0] = 12.0
With refit=True: {'data': [2.0, 4.0, 6.0], 'centered_data': [-2.0, 0.0, 2.0], 'sum': 12.0}
New mean learned: 4.0


## ObservablePipeline

Pipeline integration with automatic event emission for monitoring and debugging.

In [24]:
# Create and run an ObservablePipeline
bus = EventBus()
@bus.on(priority=0)
def global_logger(event: Event) -> None:
    print(f"[GLOBAL] -> {event.source}")

@bus.on(PipelineEvent.PIPELINE_START)
def global_logger(event: Event) -> None:
    print(f"[{event.type}] -> {event.source}")

@bus.on(PipelineEvent.PIPELINE_END)
def global_logger(event: Event) -> None:
    print(f"[{event.type}] -> {event.source}")

@bus.on(PipelineEvent.STEP_START)
def global_logger(event: Event) -> None:
    print(f"[{event.type}] -> {event.source}")

@bus.on(PipelineEvent.STEP_END)
def global_logger(event: Event) -> None:
    print(f"[{event.type}] -> {event.source}")



print("=== ObservablePipeline Demo ===")
storage = DictStorage()
pipeline = ObservablePipeline(steps=[Load(), Sum()], bus=bus, name="Demo", storage=storage)
pipeline.run()

print(f"\nFinal STATE: {storage.as_dict()}")

=== ObservablePipeline Demo ===
[GLOBAL] -> Demo
[PipelineEvent.PIPELINE_START] -> Demo
[GLOBAL] -> Demo.Load
[PipelineEvent.STEP_START] -> Demo.Load
[GLOBAL] -> Demo.Load
[PipelineEvent.STEP_END] -> Demo.Load
[GLOBAL] -> Demo.Sum
[PipelineEvent.STEP_START] -> Demo.Sum
  Sum of [1, 2, 3, 4, 5] = 15
[GLOBAL] -> Demo.Sum
[PipelineEvent.STEP_END] -> Demo.Sum
[GLOBAL] -> Demo
[PipelineEvent.PIPELINE_END] -> Demo

Final STATE: {'data': [1, 2, 3, 4, 5], 'sum': 15}


## RepeatablePipeline

Pipeline integration for repeating steps a fixed number of times eventually interrupted by a condition

`clear_storage=True` means storage is cleared at each iteration


In [25]:
@Step.needs("sum", "tot")
class Accumulate(Step):
    def bindings(self):
        return {"sum": "sum", "tot": "tot"}

    def compute(self, sum: int, tot: int = 0):
        tot += sum
        return {"tot": tot}

print("\n=== RepeatablePipeline with clear_storage=True ===")
storage = DictStorage()
pipeline = RepeatablePipeline(steps=[Load(), Sum(), Accumulate()], name="RepeatDemo", storage=storage, count=3, clear_storage=True)
pipeline.run()
print(f"\nFinal STATE: {storage.as_dict()}")


=== RepeatablePipeline with clear_storage=True ===
  Sum of [1, 2, 3, 4, 5] = 15
  Sum of [1, 2, 3, 4, 5] = 15
  Sum of [1, 2, 3, 4, 5] = 15

Final STATE: {'data': [1, 2, 3, 4, 5], 'sum': 15, 'tot': 15}


`clear_storage=True` means accumulates over iterations

In [26]:
print("\n=== RepeatablePipeline with clear_storage=False ===")
storage = DictStorage()
pipeline = RepeatablePipeline(steps=[Load(), Sum(), Accumulate()], name="RepeatDemo", storage=storage, count=3, clear_storage=False)
pipeline.run()
print(f"\nFinal STATE: {storage.as_dict()}")


=== RepeatablePipeline with clear_storage=False ===
  Sum of [1, 2, 3, 4, 5] = 15
  Sum of [1, 2, 3, 4, 5] = 15
  Sum of [1, 2, 3, 4, 5] = 15

Final STATE: {'data': [1, 2, 3, 4, 5], 'sum': 15, 'tot': 45}


A `StoragePredicate` can be used to stop the pipeline based on storage content

In [28]:
def greater_than(key: str, value: int) -> StoragePredicate:
    return lambda storage: storage.get([key])[key] > value

print("\n=== RepeatablePipeline with storage based stopping condition ===")
storage = DictStorage({"tot": 0})
pipeline = RepeatablePipeline(steps=[Load(), Sum(), Accumulate()], name="RepeatDemo", storage=storage, count=3, clear_storage=False, predicate=greater_than("tot", 20))
pipeline.run()
print(f"\nFinal STATE: {storage.as_dict()}")


=== RepeatablePipeline with storage based stopping condition ===
  Sum of [1, 2, 3, 4, 5] = 15
  Sum of [1, 2, 3, 4, 5] = 15

Final STATE: {'tot': 30, 'data': [1, 2, 3, 4, 5], 'sum': 15}


## Key Takeaways

1. **Pipeline Architecture**: Orchestrate sequences of steps with automatic data flow management
2. **Step Composition**: Define reusable components with clear input/output contracts
3. **Event Hooks**: Respond to pipeline lifecycle events for monitoring and custom behavior
4. **Priority Systems**: Control execution order with multi-priority event handlers
5. **Fittable Pipelines**: ML-specific workflows with separate fit/transform phases
6. **Observable Pipelines**: Automatic event emission for external monitoring
7. **Repeatable Pipelines**: Conditional re-execution based on storage state
